# Speech sythesis
Original paper [here](https://arxiv.org/pdf/1710.08969v1.pdf)
Github with source code [here](https://github.com/r9y9/deepvoice3_pytorch)
This repository [here]()

## Preprocessing
- Create directory for this project or just clone repository
- Create venv with python > 3.5 to escape unnecessary installations to your system
- Run this notebook

In [7]:
# clone original code
! git clone https://github.com/r9y9/deepvoice3_pytorch.git cloned_repo
! cd cloned_repo && git checkout abf0a21 && mv -f * ..
! rm -rf cloned_repo

Cloning into 'cloned_repo'...
remote: Enumerating objects: 1144, done.
remote: Total 1144 (delta 0), reused 0 (delta 0), pack-reused 1144
Receiving objects: 100% (1144/1144), 6.93 MiB | 4.85 MiB/s, done.
Resolving deltas: 100% (639/639), done.
Note: checking out 'abf0a21'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at abf0a21 dep: relax lws requirement. latest v0.2 works


In [8]:
# downloading model data
! rm -rf model_state
! mkdir model_state
! wget -q --show-progress -P model_state "https://www.dropbox.com/s/0ck82unm0bo0rxd/20180505_deepvoice3_ljspeech.json" 
! wget -q --show-progress -P model_state "https://www.dropbox.com/s/5ucl9remrwy5oeg/20180505_deepvoice3_checkpoint_step000640000.pth"

20180505_deepvoice3 100%[===================>]   1,75K  --.-KB/s    in 0s      
20180505_deepvoice3 100%[===================>] 151,89M  10,8MB/s    in 15s     


In [1]:
! pip install -r requirements.txt --quiet

In [12]:
# ! pip install -e ".[bin]"

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/silence/Downloads/Article_2/speech_generation_demo
  Attempting uninstall: deepvoice3-pytorch
    Found existing installation: deepvoice3-pytorch 0.0.6+abf0a21
    Uninstalling deepvoice3-pytorch-0.0.6+abf0a21:
      Successfully uninstalled deepvoice3-pytorch-0.0.6+abf0a21
  Running setup.py develop for deepvoice3-pytorch
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
# Need to process text-to-speech (some additional installation)
import nltk
! python -m nltk.downloader cmudict
! python -m nltk.downloader punkt

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package cmudict to /home/silence/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /home/silence/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import train
import synthesis
import IPython
from deepvoice3_pytorch import frontend
import IPython
from IPython.display import Audio
import hparams
import json

/home/silence/Downloads/Article_2/speech_generation_demo/venv/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/silence/Downloads/Article_2/speech_generation_demo/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/silence/Downloads/Article_2/speech_generation_demo/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; 

In [4]:
# define options
hparams_path = 'model_state/20180505_deepvoice3_ljspeech.json'
checkpoint_path = 'model_state/20180505_deepvoice3_checkpoint_step000640000.pth'
max_decoder_steps = 500
replace_pronunciation_prob = 0.0

In [5]:
# Load parameters from preset
with open(hparams_path) as f:
  hparams.hparams.parse_json(f.read())
  
# Inject frontend text processor
synthesis._frontend = getattr(frontend, "en")
train._frontend =  getattr(frontend, "en")

# alises
fs = hparams.hparams.sample_rate
hop_length = hparams.hparams.hop_size

In [6]:
def say_text(model, text, p=replace_pronunciation_prob, speaker_id=None, fast=True):
  from synthesis import tts as _tts
  waveform, alignment, spectrogram, mel = _tts(model, text, p, speaker_id, fast)
  IPython.display.display(Audio(waveform, rate=fs))

In [7]:
from train import build_model
from train import restore_parts, load_checkpoint
model = build_model()
model = load_checkpoint(checkpoint_path, model, None, True)
model.seq2seq.decoder.max_decoder_steps = max_decoder_steps

Load checkpoint from: model_state/20180505_deepvoice3_checkpoint_step000640000.pth


In [8]:
say_text(model, 'Hello. I am really alive!')

/home/silence/Downloads/Article_2/speech_generation_demo/venv/lib/python3.6/site-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/silence/Downloads/Article_2/speech_generation_demo/venv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)


In [9]:
say_text(model, "There is a Hell Believe Me I've Seen It. There is a Heaven Let's Keep it a Secret")